In [16]:
import pandas as pd
import os
import numpy as np

def custom_timedelta(t):
    try:
        td = pd.to_timedelta(t)
        return round(td.total_seconds(), 0)
    except ValueError:
        dot_count = t.count('.')
        if dot_count == 1:
            days, time = t.split('.')
            try:
                sign = -1 if days.startswith('-') else 1
                days = abs(int(days))
                hours, minutes, seconds = time.split(':')
                total_seconds = sign * ((days * 24 * 3600) + (int(hours) * 3600) + (int(minutes) * 60) + round(float(seconds), 0))
                return round(total_seconds, 0)
            except ValueError:
                raise ValueError("Invalid time format, expected either hh:mm:ss or d.hh:mm:ss")
        elif dot_count == 2:
            days_time, fraction = t.rsplit('.', 1)
            days, time = days_time.split('.')
            try:
                sign = -1 if days.startswith('-') else 1
                days = abs(int(days))
                hours, minutes, seconds = time.split(':')
                seconds = float(seconds) + round(float(f"0.{fraction}"), 0)
                total_seconds = sign * ((days * 24 * 3600) + (int(hours) * 3600) + (int(minutes) * 60) + seconds)
                return round(total_seconds, 0)
            except ValueError:
                raise ValueError("Invalid time format, expected either hh:mm:ss or d.hh:mm:ss")
        else:
            raise ValueError("Invalid time format, expected either hh:mm:ss or d.hh:mm:ss")

# Initialize a list to store the metrics
metrics_list = []

def apply_transformations(file_path):

    # Read the CSV file
    data = pd.read_csv(file_path)

    # Convert date columns to datetime, handling errors
    datetime_cols = ['scheduled_arrival', 'scheduled_departure', 'arrival', 'departure']
    for col in datetime_cols:
        data[col] = pd.to_datetime(data[col], errors='coerce')



    ###################################################################
    # Sorting data correctly. We cannot fully rely on sort_time, since sometimes the order of sequence_number 
    # does not match sort time. That's why it is necessary to sort as done below. 
    ###################################################################

    # Create and sort 'sort_time' column
    data['sort_time'] = data['arrival'].fillna(data['departure'])

    # Group by 'train_number' and 'trainpart_id', and find the earliest 'sort_time' for each group.
    earliest_sort_times = data.groupby(['train_number', 'trainpart_id'])['sort_time'].min().reset_index()

    # Create a rank for each 'trainpart_id' within each 'train_number' based on the earliest 'sort_time'.
    earliest_sort_times['trainpart_id_order'] = earliest_sort_times.groupby('train_number')['sort_time'].rank(method='first').astype(int)

    # Merge this order back into the original dataframe.
    data = data.merge(earliest_sort_times[['train_number', 'trainpart_id', 'trainpart_id_order']], on=['train_number', 'trainpart_id'], how='left')

    # Now sort the dataframe by 'train_number', 'trainpart_id_order', and 'sequence_number'.
    data.sort_values(by=['train_number', 'trainpart_id_order', 'sequence_number'], inplace=True)

    # Generate the 'train_unique_sequence' 
    data['train_unique_sequence'] = data.groupby(['train_number']).cumcount() + 1

    ###################################################################
    ###################################################################
    ###################################################################



    # Necessary - changing operator class to numeric and then to integers
    data['operator_class'] = pd.to_numeric(data['operator_class'], errors='coerce')

    # Fill NaN values if any resulted from coercion, then convert to integers
    # Here, 0 is used as a placeholder, but you may choose a different value as appropriate
    data['operator_class'] = data['operator_class'].fillna(0).astype(int)

    

    ####################################################################
    # We need for all further steps only one traction unit, so we filter out any unnecessary traction units
    ####################################################################


    # Set the flag to True for each row initially
    data['lowest_unique_sequence_flag'] = True

    # Flag adjustment for multiple traction units
    multiple_units = data['number_of_traction_units'] > 1 

    group_min_sequence = data[multiple_units].groupby(['sort_time', 'db640_code', 'ocp_type', 'trainpart_id'])['train_unique_sequence'].transform('min')
    data.loc[multiple_units, 'lowest_unique_sequence_flag'] = data[multiple_units]['train_unique_sequence'] == group_min_sequence
    data = data[data['lowest_unique_sequence_flag']]

    
    ###########################################################
    # Adjust arrival times for initial delays in freight trains
    ###########################################################

    data = adjust_arrival_for_initial_delays_in_freight(data)


    #########################################################################################
    # Map operator class and category to integers and export it to dictionaries for later use
    #########################################################################################

    data = map_columns_to_integers(data)


    ##########################################################
    # Creating time features suitable for deep learning models
    ##########################################################

    data = create_sin_cos_time_features(data)

    data = creating_continuous_day_feature(data)

    #################


    # Convert delay times to timedelta and seconds

    data['departure_delay_seconds'] = data['departure_delay_in_seconds'].apply(custom_timedelta)

    data['arrival_delay_seconds'] = data['arrival_delay_in_seconds'].apply(custom_timedelta)

    data['primary_delay'] = data['primary_delay'].apply(custom_timedelta)



    # Sort and reset index
    data.sort_values(by=['train_number', 'train_unique_sequence'], inplace=True)
    data.reset_index(drop=True, inplace=True)

    
    columns_to_keep = [
    'operator_class', 
    'train_number', 
    'ocp_type',
    'arrival', 
    'departure', 
    'train_unique_sequence',
    'number_of_traction_units',  
    'freight', 
    'trainpart_weight', 
    'longitude', 
    'latitude',
    'category',
    'trainpart_id',
    'departure_delay_seconds', 
    'arrival_delay_seconds',
    'sequence_number',
    'db640_code',
    'sin_seconds_since_midnight_departure',
    'cos_seconds_since_midnight_departure',
    'sin_seconds_since_midnight_arrival',
    'cos_seconds_since_midnight_arrival',
    'day_value',
    'primary_delay',
    'task_id'
    ]

    # Set negative delay_seconds to 0 - otherwise we also get delay jumps from negative to negative delay
    data['departure_delay_seconds'] = data['departure_delay_seconds'].clip(lower=0)
    data['arrival_delay_seconds'] = data['arrival_delay_seconds'].clip(lower=0)


    data = data[columns_to_keep]

    data.sort_values(by=['train_number', 'train_unique_sequence'], inplace=True)

    # Important - the delay jump departure is the delay increase, which is in the same rows as the PD 
    data['delay_jump'] = data.groupby('train_number')['departure_delay_seconds'].diff().fillna(data['departure_delay_seconds']).clip(lower=0)

    data['secondary_delay'] =  (data['delay_jump'] - data['primary_delay']).clip(lower=0)
    data['primary_delay_label'] = data['primary_delay'] >= data['secondary_delay']


    # Step 4: Create label for delay jump above 60 seconds
    data['nodes_to_classify'] = data['delay_jump'] > 60
    

    ############################################################################
    # Compute statistics regarding class labels 
    ############################################################################

    compute_statistics(data)

    ############################################################################
    ############################################################################


    # Forward fill NaN in longitude and latitude for the same train number
    data['longitude'] = data.groupby('train_number')['longitude'].fillna(method='ffill')
    data['latitude'] = data.groupby('train_number')['latitude'].fillna(method='ffill')

    # Backward fill remaining NaN in longitude and latitude for the same train number
    data['longitude'] = data.groupby('train_number')['longitude'].fillna(method='bfill')
    data['latitude'] = data.groupby('train_number')['latitude'].fillna(method='bfill')
    

    # Check for duplicates in the 'scheduledID' column
    if data['task_id'].duplicated().any():
        print("Warning: 'scheduledID' does not uniquely identify each row.")
    else:
        print("'task_id' uniquely identifies each row.")

    # 'sequence_number', 'trainpart_id', 
    missing_values = data.isnull().sum()

    print(missing_values)

    return data

In [17]:
def compute_statistics(data):
    # Calculating metrics for overall data
    total_nodes = len(data)
    nodes_to_classify = data['nodes_to_classify'].sum()
    percentage_nodes_to_classify = (nodes_to_classify / total_nodes) * 100
    primary_delay_label_true = data[data['nodes_to_classify']]['primary_delay_label'].sum()
    percentage_primary_delay_label_true = (primary_delay_label_true / nodes_to_classify) * 100
    relevant_nodes = data[data['nodes_to_classify']]
    instances_both_conditions = ((relevant_nodes['secondary_delay'] > 60) & (relevant_nodes['primary_delay'] > 60)).sum()
    percentage_both_conditions = (instances_both_conditions / len(relevant_nodes)) * 100

    print("Overall Metrics:")
    print("Percentage of nodes to classify: ", percentage_nodes_to_classify)
    print("Percentage of nodes to classify where primary_delay_label is True: ", percentage_primary_delay_label_true)
    print("Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes: ", percentage_both_conditions)

    # Append overall metrics to the metrics list
    metrics_list.append({
        'type': 'overall',
        'percentage_nodes_to_classify': percentage_nodes_to_classify,
        'percentage_primary_delay_label_true': percentage_primary_delay_label_true,
        'percentage_both_conditions': percentage_both_conditions
    })

    # Calculating metrics for freight trains
    freight_data = data[data['freight'] == True]
    total_nodes_freight = len(freight_data)
    nodes_to_classify_freight = freight_data['nodes_to_classify'].sum()
    percentage_nodes_to_classify_freight = (nodes_to_classify_freight / total_nodes_freight) * 100
    primary_delay_label_true_freight = freight_data[freight_data['nodes_to_classify']]['primary_delay_label'].sum()
    percentage_primary_delay_label_true_freight = (primary_delay_label_true_freight / nodes_to_classify_freight) * 100
    relevant_nodes_freight = freight_data[freight_data['nodes_to_classify']]
    instances_both_conditions_freight = ((relevant_nodes_freight['secondary_delay'] > 60) & (relevant_nodes_freight['primary_delay'] > 60)).sum()
    percentage_both_conditions_freight = (instances_both_conditions_freight / len(relevant_nodes_freight)) * 100

    print("Freight Metrics:")
    print("Percentage of nodes to classify (freight): ", percentage_nodes_to_classify_freight)
    print("Percentage of nodes to classify where primary_delay_label is True (freight): ", percentage_primary_delay_label_true_freight)
    print("Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes (freight): ", percentage_both_conditions_freight)

    # Append freight metrics to the metrics list
    metrics_list.append({
        'type': 'freight',
        'percentage_nodes_to_classify': percentage_nodes_to_classify_freight,
        'percentage_primary_delay_label_true': percentage_primary_delay_label_true_freight,
        'percentage_both_conditions': percentage_both_conditions_freight
    })

    # Calculating metrics for passenger trains
    passenger_data = data[data['freight'] == False]
    total_nodes_passenger = len(passenger_data)
    nodes_to_classify_passenger = passenger_data['nodes_to_classify'].sum()
    percentage_nodes_to_classify_passenger = (nodes_to_classify_passenger / total_nodes_passenger) * 100
    primary_delay_label_true_passenger = passenger_data[passenger_data['nodes_to_classify']]['primary_delay_label'].sum()
    percentage_primary_delay_label_true_passenger = (primary_delay_label_true_passenger / nodes_to_classify_passenger) * 100
    relevant_nodes_passenger = passenger_data[passenger_data['nodes_to_classify']]
    instances_both_conditions_passenger = ((relevant_nodes_passenger['secondary_delay'] > 60) & (relevant_nodes_passenger['primary_delay'] > 60)).sum()
    percentage_both_conditions_passenger = (instances_both_conditions_passenger / len(relevant_nodes_passenger)) * 100

    print("Passenger Metrics:")
    print("Percentage of nodes to classify (passenger): ", percentage_nodes_to_classify_passenger)
    print("Percentage of nodes to classify where primary_delay_label is True (passenger): ", percentage_primary_delay_label_true_passenger)
    print("Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes (passenger): ", percentage_both_conditions_passenger)

    # Append passenger metrics to the metrics list
    metrics_list.append({
        'type': 'passenger',
        'percentage_nodes_to_classify': percentage_nodes_to_classify_passenger,
        'percentage_primary_delay_label_true': percentage_primary_delay_label_true_passenger,
        'percentage_both_conditions': percentage_both_conditions_passenger
    })

In [18]:
def map_columns_to_integers(data):

    # Check if the mapping dictionaries already exist in the current working directory
    if os.path.exists('category_mapping.csv') and os.path.exists('operator_class_mapping.csv'):
        # Load the existing mapping dictionaries from CSV files
        category_mapping = pd.read_csv('category_mapping.csv', index_col='category').to_dict()['category_id']
        operator_class_mapping = pd.read_csv('operator_class_mapping.csv', index_col='operator_class').to_dict()['operator_class_id']
    else:
        # Create new mapping dictionaries
        category_mapping = {category: i for i, category in enumerate(data['category'].unique(), start=1)}
        operator_class_mapping = {operator_class: i for i, operator_class in enumerate(data['operator_class'].unique(), start=1)}

        # Export the new mapping dictionaries to CSV files in the current working directory
        category_mapping_df = pd.DataFrame.from_dict(category_mapping, orient='index', columns=['category_id'])
        category_mapping_df.index.name = 'category'
        category_mapping_df.to_csv('category_mapping.csv')

        operator_class_mapping_df = pd.DataFrame.from_dict(operator_class_mapping, orient='index', columns=['operator_class_id'])
        operator_class_mapping_df.index.name = 'operator_class'
        operator_class_mapping_df.to_csv('operator_class_mapping.csv')


    # Map the values in the DataFrame using the mapping dictionaries
    data['category'] = data['category'].map(category_mapping)
    data['operator_class'] = data['operator_class'].map(operator_class_mapping)

    return data

In [19]:
def create_sin_cos_time_features(data):

    # Calculate the total seconds since the start of the day for both 'departure' and 'arrival'
    data['seconds_since_midnight_departure'] = (
        data['departure'] - data['departure'].dt.normalize()
    ).dt.total_seconds()
    data['seconds_since_midnight_arrival'] = (
        data['arrival'] - data['arrival'].dt.normalize()
    ).dt.total_seconds()

    # Apply sine and cosine transformations for cyclical encoding
    data['sin_seconds_since_midnight_departure'] = np.sin(2 * np.pi * data['seconds_since_midnight_departure'] / 86400)
    data['cos_seconds_since_midnight_departure'] = np.cos(2 * np.pi * data['seconds_since_midnight_departure'] / 86400)
    data['sin_seconds_since_midnight_arrival'] = np.sin(2 * np.pi * data['seconds_since_midnight_arrival'] / 86400)
    data['cos_seconds_since_midnight_arrival'] = np.cos(2 * np.pi * data['seconds_since_midnight_arrival'] / 86400)

    
    data = data.drop(['seconds_since_midnight_departure', 'seconds_since_midnight_arrival'], axis=1)

    return data


In [20]:
def creating_continuous_day_feature(data):
    # Ensure 'arrival' is in datetime format
    data['arrival'] = pd.to_datetime(data['arrival'])

    # Extract just the date part
    data['arrival_date'] = data['arrival'].dt.date

    # Get the sorted unique dates
    unique_days = sorted(data['arrival_date'].unique())

    # Compute scaled day values
    x_min, x_max = 0, len(unique_days) - 1  # Indices of the first and last days
    y_min, y_max = -1, 1  # Target range for scaling

    # Create a dictionary mapping each day to its scaled value
    day_mapping = {day: y_min + ((idx - x_min) * (y_max - y_min) / (x_max - x_min)) for idx, day in enumerate(unique_days)}

    # Map the dates in 'arrival_date' to their corresponding scaled value
    data['day_value'] = data['arrival_date'].map(day_mapping)

    # Drop the 'arrival_date' column if no longer needed
    data = data.drop(['arrival_date'], axis=1)

    return data

In [21]:
# This function adjusts sets the arrival column to the same value as the departure column for the initial delays of freight trains
# This is necessary, since our initial delays are sampled in the station, and in the historical data the arrival column would also already include
# the initial delay. If we would not set this, our GNN would learn this pattern and predict worse on the historical data. 

def adjust_arrival_for_initial_delays_in_freight(df):

    # Get unique train numbers for freight trains
    train_numbers = df.loc[df['freight'] == True, 'train_number'].unique()

    # Iterate over each freight train number
    for train_num in train_numbers:
        # Set arrival to departure for freight trains with train_unique_sequence 1 and ocp_type "Stop"
        mask = (df['freight'] == True) & (df['train_number'] == train_num) & (df['train_unique_sequence'] == 1) & (df['ocp_type'] == 'Stop')
        df.loc[mask, 'arrival'] = df.loc[mask, 'departure']
        df.loc[mask, 'arrival_delay_in_seconds'] = df.loc[mask, 'departure_delay_in_seconds']


    return df

In [22]:
def process_all_files(directory_path):
    # Create the export directory if it does not exist
    export_directory = os.path.join(directory_path, "export")
    if not os.path.exists(export_directory):
        os.makedirs(export_directory)

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            transformed_data = apply_transformations(file_path)

            # Define the output file path
            output_file_name = 'transformed_' + file_name
            output_file_path = os.path.join(export_directory, output_file_name)

            # Export the transformed DataFrame to a new CSV file
            transformed_data.to_csv(output_file_path, index=False)
            print(f"Transformed data exported to {output_file_path}")

# Specify the directory containing the CSV files
data_directory = 'data'

# Process all CSV files in the directory
process_all_files(data_directory)

/var/folders/w2/jctqt2jx2g926tqh21c367bm0000gn/T/ipykernel_57981/624504691.py:42: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Overall Metrics:
Percentage of nodes to classify:  3.071153635441445
Percentage of nodes to classify where primary_delay_label is True:  94.57935320520198
Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes:  1.7723558522269538
Freight Metrics:
Percentage of nodes to classify (freight):  6.818324115621413
Percentage of nodes to classify where primary_delay_label is True (freight):  95.86776859504133
Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes (freight):  1.8977655341291706
Passenger Metrics:
Percentage of nodes to classify (passenger):  2.3071555010893245
Percentage of nodes to classify where primary_delay_label is True (passenger):  93.80302471412763
Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes (passenger):  1.6967908520841017
'task_id' uniquely identifies each row.
operator_class                          0
tra

In [23]:
# Calculate average metrics
average_metrics = {
    'percentage_nodes_to_classify': np.mean([metric['percentage_nodes_to_classify'] for metric in metrics_list]),
    'percentage_primary_delay_label_true': np.mean([metric['percentage_primary_delay_label_true'] for metric in metrics_list]),
    'percentage_both_conditions': np.mean([metric['percentage_both_conditions'] for metric in metrics_list])
}

# Print average metrics
print("Average Percentage of nodes to classify: ")
print(average_metrics['percentage_nodes_to_classify'])
print("Average Percentage of nodes to classify where primary_delay_label is True: ")
print(average_metrics['percentage_primary_delay_label_true'])
print("Average Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes: ")
print(average_metrics['percentage_both_conditions'])

Average Percentage of nodes to classify: 
4.065544417384062
Average Percentage of nodes to classify where primary_delay_label is True: 
94.75004883812365
Average Percentage of instances where both secondary_delay > 60 and primary_delay > 60 relative to relevant nodes: 
1.788970746146742
